In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
import spacy
from spacy.language import Language
import re
import hjson

In [2]:
model_path="../models/llama-3-sqlcoder-8b"

model=AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto", max_memory={0: "3800MB"})
tokenizer=AutoTokenizer.from_pretrained(model_path)
print(torch.cuda.is_available())

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/josafat/Documentos/bajaware-ia/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/josafat/Documentos/bajaware-ia/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the disk.


True


In [3]:
query="De los CLAVE_CONTRATO de esos CLAVE_CLIENTE, quienes tienen el reporte CLAVE_REP MX_SOF_R28A_2814_00 en su CLAVE_CONTRATO?"
print(query)

De los CLAVE_CONTRATO de esos CLAVE_CLIENTE, quienes tienen el reporte CLAVE_REP MX_SOF_R28A_2814_00 en su CLAVE_CONTRATO?


In [4]:
text=f"""### Instruction:
Your task is to generate valid duckdb SQL to answer the following question, given a duckdb database schema.

### Input:
Here is the database schema that the SQL query will run on:
CREATE TABLE CONTRATOS_REPORTES (
    ID_CONTRATO_REPORTES INT NOT NULL, -- Identificador único del contrato-reportes
    CLAVE_CONTRATO VARCHAR(50) NOT NULL, -- Clave del contrato (relacionada con la tabla CONTRATOS)
    CLAVE_REP VARCHAR(100) NOT NULL, -- Clave del reporte (relacionada con la tabla INVENTARIO_REPORTES)
    FECHA_ESTIMADA_QA DATE NULL, -- Fecha estimada para QA
    FECHA_INSTALADO_QA DATE NULL, -- Fecha de instalación en QA
    FECHA_ESTIMADA_CERT DATE NULL, -- Fecha estimada para certificación
    FECHA_CERTIFICADO DATE NULL, -- Fecha de certificación
    FECHA_ESTIMADA_PROD DATE NULL, -- Fecha estimada para producción
    FECHA_INSTALADO_PROD DATE NULL, -- Fecha de instalación en producción
    ETAPA INT NULL, -- Etapa del contrato (puede ser un número entero)
    EN_USO VARCHAR(2) NULL, -- Indicador de si el contrato está en uso (puede ser 'SI' o 'NO')
    CONSTRAINT PK_CONTRATOS_REPORTES PRIMARY KEY (CLAVE_CONTRATO, CLAVE_REP) -- Clave primaria que garantiza la unicidad del contrato-reportes
)

CREATE TABLE CONTRATOS (
    ID_CONTRATO INT NOT NULL, -- Identificador único del contrato
    CLAVE_CONTRATO VARCHAR(50) NOT NULL, -- Clave del contrato
    NOMBRE_CONTRATO VARCHAR(100) NOT NULL, -- Nombre del contrato
    FECHA_ALTA DATE NOT NULL, -- Fecha de alta del contrato
    FECHA_MODIFICA DATE NOT NULL, -- Fecha de ultima modificación del contrato
    CLAVE_CLIENTE VARCHAR(50) NULL, -- Fecha de cliente
    CLAVE_PLATAFORMA VARCHAR(10) NULL, -- Clave de la plataforma java o net
    CONSTRAINT PK_CONTRATOS PRIMARY KEY (CLAVE_CONTRATO) -- Clave primaria que garantiza la unicidad del contrato
)

### Question:
{query}

### Response (use duckdb shorthand if possible):"""

In [5]:
inputs=tokenizer(text, return_tensors="pt")
input_ids=inputs["input_ids"]
attention_mask=inputs["attention_mask"]

generated_ids=model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=256)
response=(tokenizer.decode(generated_ids[0], skip_special_tokens=True)).replace(text, "")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/josafat/Documentos/bajaware-ia/venv/lib/python3.11/site-packages/transformers/generation/utils.py:2345: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 SELECT c.clave_contrato FROM contratos c JOIN contratos_reportes cr ON c.clave_contrato = cr.clave_contrato WHERE cr.clave_rep = 'MX_SOF_R28A_2814_00';
